# ReactのContext

- 2018/11/14 勉強会

# アジェンダ

1. ContextAPI概要
1. いつ使うか
1. Reduxとの応対

# ContextAPI概要



## 従来のReact

- 親から子へデータを渡すときはpropsを使う

```
// 親
export default class Parent extends React.Componet {
  render() {
    const count = 1;
    // `count`を子孫にわたす
    return <Child count={count} />;
  }
}

// 曾孫 やっとここで使いたい
const Child = ({ count }) => <div>count: {count}</div>;
```

### もっと入れ子&複数propsになるとき

- 親→子→孫→..渡すときにバケツリレーが続いてしまう
    - これを`Prop Drilling`問題という
    
```
export default class Parent extends React.Componet {
  render() {
    const count = 1;
    const word = 'hello';

    return <Child count={count} word={word} />;
  }
}

// 子 ここでは使わない
const Child = ({ count, word }) => <Grandson count={count} word={word} />;

// 孫 ここでも使わない
const Grandson = ({ count, word }) => (
  <GreatGrandchild count={count} word={word} />
);

// 曾孫 やっとここで使いたい
const GreatGrandchild = ({ count, word }) => (
  <>
    <div>count: {count}</div>
    <div>word: {word}</div>
  </>
);
```

## 従来の解決の仕方

- Reduxを使う
- spread attibutesを使う
    - {...props}みたいなやつ

## ContextAPIとは

- React製
- contextを使って子孫にデータを送る
- `this.props`の代わりに`this.context`とかを使う??????????



contextAPI

https://reactjs.org/docs/context.html

### 使い所


## ContextAPIを用いた最小の例

```
import React, { createContext } from 'react';

const { Provider, Consumer } = createContext(1);

export const Parent = () => (
  <Provider value={{ count: 1 }}>
    <Child />
  </Provider>
);

// 子 propsを伝搬させてない
const Child = () => <Grandson />;

// 孫 同じく伝搬させてない
const Grandson = () => <GreatGrandchild />;

const GreatGrandchild = () => (
  <Consumer>{({ count }) => <div>values: {count}</div>}</Consumer>
);
```



### 解説

#### [React.createContext](https://reactjs.org/docs/context.html#reactcreatecontext)

- ProviderとConsumerをペアで作る
- 上記の例のように2値を一気に受け取るものもアレば以下のような使い方もある
```
const Root = React.createContext;
...
<Root.Provider />
...
<Root.Consumer />
```

#### [Provider](https://reactjs.org/docs/context.html#contextprovider)

- Consumerにcontextを渡すコンポーネント
- `value`を使って渡す
- 値だけでなく関数も渡せる
- (ReduxのProviderと全く同じ名前なのは..)

#### [Consumer](https://reactjs.org/docs/context.html#contextconsumer)

- Providerからcontextを受け取る
- 中には関数を書く
- 主にclassではなくFCの中で使う？？？？？？？？？？？
    - classでも使えるの？？？？？？？？？？？・
- 子に関数をもつ
```
<Consumer>
  {({ count }) => <div>values: {count}</div>}
</Consumer>
```
    

## ちなみに公式では

- テーマカラーやログイン中ユーザーなど、状態が頻繁に変わるわけではないが、いろんなコンポーネントからアクセスされるやつをContextに渡している

## 少し拡張する

- 次の例ではまだ紹介していない`Class.contextType`も使う
- 関数も渡す



```
import React from 'react';

// store的ななにか
const store = {
  num: 0,
  increment: () => {}
};

// contextを作る
const CounterContext = React.createContext(store);

// 親
export default class Parent extends React.Component {
  constructor(props) {
    super(props);
    this.state = {
      num: store.num
    };
  }

  increment = () => {
    this.setState({
      num: this.state.num + 1
    });
  };

  render() {
    return (
      <CounterContext.Provider
        value={{ num: this.state.num, increment: this.increment }}
      >
        <p>num</p>
        <Child />
      </CounterContext.Provider>
    );
  }
}

// 子
const Child = () => <Grandson />;

// 孫
// これはclass
class Grandson extends React.Component {
  static contextType = CounterContext;
  render() {
    const { num, increment } = { ...this.context };
    return (
      <>
        <p>{num}</p>
        <button onClick={increment}>increment</button>
      </>
    );
  }
}

```

### 解説

- [Class.contextType](https://reactjs.org/docs/context.html#classcontexttype)

- 前の例と同じように、Childコンポーネントをではなにもしていない
- `value`で値と一緒に関数も渡している
```
<CounterContext.Provider
  value={{ num: this.state.num, increment: this.increment }}
>
  <p>num</p>
  <Child />
</CounterContext.Provider>
```

- 孫コンポーネントの以下の部分でcontextを受け取っている
- `static contextTypes = CounterContext`
- 受け取ったcontextは`this.context.hogehoge`で利用できる
- 受け取るコンポーネントがclassならcontextTypesを使い、SFCならConsumerを使う(?)

### 上の例の孫を以下のようにSFCに変えても動く

```
const Grandson = () => (
  <CounterContext.Consumer>
    {({ num, increment }) => (
      <>
        <p>{num}</p>
        <button onClick={increment}>increment</button>
      </>
    )}
  </CounterContext.Consumer>
);

```


- createContext()の引数=初期値がよくわからん


### Caveats とは


## Reduxとの応対

- 今後もReduxを使っていくが
- 小規模だったり、ギリReduxを使いたくないぐらいのときに使えそう
- それ以上の規模のときに使うとぐちゃりそう

- ReduxはReactの旧Context機能を使って実装されている

## Hooksを使う

- 親もFCでよくなる
- 以下の例は親以外は一緒

```
import React, { useState, useContext } from 'react';

// store的ななにか
const store = {
  num: 0,
  increment: () => {}
};

// contextを作る
const CounterContext = React.createContext(store);

// 親
const Parent = () => {
  useContext(CounterContext);
  const [num, setNum] = useState(0);

  return (
    <CounterContext.Provider
      value={{
        num,
        increment: () => {
          setNum(num + 1);
        }
      }}
    >
      <Child />
    </CounterContext.Provider>
  );
};
export default Parent;

// 子
const Child = () => <Grandson />;

// 孫
const Grandson = () => (
  <CounterContext.Consumer>
    {({ num, increment }) => (
      <>
        <p>{num}</p>
        <button onClick={increment}>increment</button>
      </>
    )}
  </CounterContext.Consumer>
);
```




### 解説

- useContextの使い方


## ContextAPI + react-redux

先日のリリース  
https://github.com/reduxjs/react-redux/releases/tag/v6.0.0-beta.1

`npm install react-redux@next`で入れる  
`npm install --save redux` reduxも  


- testを参考に
https://github.com/reduxjs/react-redux/blob/85fb553ba8e3f4b0efc158d2e48aafb4c18a04d4/test/components/Provider.spec.js


https://medium.com/@terrierscript/react%E3%81%AE%E6%96%B0context-api%E3%81%A8redux-is-dead%E3%81%AF%E3%81%A9%E3%81%86%E9%96%A2%E4%BF%82%E3%81%99%E3%82%8B%E3%81%AE%E3%81%8B-6d12a32f2f0c


Consuming Multiple Contexts

https://reactjs.org/docs/context.html#consuming-multiple-contexts
複雑性が増しそう

# 参考

- [React v16で実装された new Context APIを使って、Reduxへ別れを告げる - Qiita](https://qiita.com/loverails/items/50126e874b24ff984471)
- [【React - ContextAPI】Consumerの正体は、イケイケなコンポーネントだった - Qiita](https://qiita.com/kazumicho/items/4f862b618eda09e14ec7)
- [Redux と React Context API - 適材適所のススメ - Qiita](https://qiita.com/ossan-engineer/items/c3e5bd4d9bb4db04f80d)
- [React v16で実装された new Context APIを使って、Reduxへ別れを告げる② - Qiita](https://qiita.com/loverails/items/b8dc97384951f785322d#_reference-4cce66b31b1c7f0abaa4)
- 未読[Context APIを活用して、Reduxのstate管理から外したAccordionコンポーネントを作る - Qiita](https://qiita.com/rhirayamaaan/items/d9f7c62c4b5804eba74d#_reference-a799b37f02668b9c7173)

## reduxっぽいやつ(未完成)

- https://qiita.com/zaburo/items/ce46e4a53e23ef210df1

```
import React from 'react';

// store的ななにか
const store = new class {
  state = {
    num: 0
  };
  actions = fn => {
    console.log(this.state);
    this.state = fn(this.state);
  };
  // actions: fn => {
  //   this.state = fn(this.state);
  // }
  // actions: {
  //   increment: () => {
  //     this.state.num += 1;
  //   }
  // }
}();

// contextを作る
const CounterContext = React.createContext({ num: 0 });

// 親
export default class Parent extends React.Component {
  render() {
    return (
      <CounterContext.Provider value={store}>
        <p>parenat: {store.state.num}</p>
        <p>num</p>
        <Child />
      </CounterContext.Provider>
    );
  }
}

// 子
const Child = () => <Grandson />;

// 孫
// // これはclass
class Grandson extends React.Component {
  static contextType = CounterContext;
  render() {
    const { state, actions } = { ...this.context };
    console.log(state);
    console.log(actions);
    return (
      <>
        <p>{state.num}</p>
        <button
          onClick={() =>
            actions(s => {
              // console.log(s);
              return {
                ...s,
                num: s.num + 1
              };
            })
          }
        >
          increment
        </button>
      </>
    );
  }
}

// Consumer使う版
// const Grandson = () => (
//   <CounterContext.Consumer>
//     {({ state, actions }) => (
//       <div>
//         <p>{state.num}</p>
//         <button
//           onClick={() =>
//             actions(s => {
//               console.log(s);
//               return {
//                 ...s,
//                 num: s.num += 1
//               };
//             })
//           }
//         >
//           increment
//         </button>
//       </div>
//     )}
//   </CounterContext.Consumer>
// );
```
